In [52]:
import numpy as np

# Define initial parameters
beta = 0.99 # D factor
sigma = 1 # Risk averson
gamma = 0.5 # Price stickines parameter
delta = 0.1  # Depreciation rate
alpha = 0.5 # Degree of monopolistic competition
theta = 0.75 # Degree of price index
phi = 0.5 # Inflation target
rho_a = 0.9 # Persistence of technology shock
sigma_a = 0.01 # Standard deviation of technology shock
sigma_b = 0.01 # Standard deviation of technology shock


# Define initial variables
T = 100 # Number of periods
y = np.zeros(T) # Output
k = np.zeros(T+1) #Kapital shock
p = np.zeros(T+1) #output by the firm
a = np.zeros(T+1)
c = np.zeros(T) # Consumption
i = np.zeros(T) # Investment
pi = np.zeros(T) # Inflation
w = np.zeros(T) # Real wage
r = np.zeros(T) # Real interest rate

In [53]:
def productivity_shock(self, T):
    epsilon = np.random.normal(scale=self.sigma_b, size=T)
    return np.exp(epsilon)

In [54]:
def labor_supply(self, w, r, price_level, output, productivity):
    marginal_utility = self.gamma * price_level * output / (1 - self.alpha)
    return (w * marginal_utility) / (self.delta * productivity * price_level)

In [55]:
# Define the household optimization function
def household_optimization(w, r, pi, a):
    # Compute the optimal consumption and investment levels
    c_star = (1 - gamma) * (w + (r - delta) * a) + pi * gamma * a - w
    i_star = a - (1 - delta) * a - c_star
    
    # Check for 0 denominators and set optimal consumption and investment to a small positive number in those cases
    if w == 0 or r == delta or c_star == 0 or i_star == 0:
        c_star = 1e-10
        i_star = 1e-10
    
    return c_star, i_star


In [56]:
# Define firms' optimization problem
# This is all from ChatGPT ***TO BE CHECKED
def firm_optimization(w, a):
    # Define profit function
    def profit(q):
        return (w / alpha) * (q**(1 + alpha)) - a * q
    
    # Solve optimization problem
    from scipy.optimize import minimize_scalar
    res = minimize_scalar(profit)
    q_star = res.x
    p_star = w * (q_star**alpha)
    
    return p_star

In [57]:
def investment_decision(r, y, k, w):
    delta = 0.1   # depreciation rate
    phi = 2       # adjustment cost parameter
    alpha = 0.3   # capital share in production function
    inv = phi / (1 + phi) * (alpha * y / (r + delta)) + (1 / (1 + phi)) * k + w
    return inv

In [58]:
# Defining simple central bank policy rule
# Later I should check for more advanced rules
def central_bank_policy_rule(pi, y):
    i_star = (1/beta) * ((1 + r) / (1 + pi)) * (1 / (1 + y))
    pi_star = theta * pi + (1 - theta) * phi + gamma * (y - 1)
    r_star = (1 / beta) * (1 + pi_star) / (1 + pi) - 1
    
    return i_star, pi_star, r_star

In [59]:
# Define initial conditions
a[0] = 1
k[0] = 1
w[0] = 1e-10 #w to a small positive number cus' i get divided by zero error
r[0] = 0.01
pi[0] = 1e-10 # change initial value of pi to a small positive number
y[0] = 1
c[0], i[0] = household_optimization(w[0], r[0], pi[0], a[0])
p[0] = firm_optimization(w[0], a[0])


In [60]:
def simulate_economy(T, alpha, beta, gamma, delta, sigma_a, sigma_b, pi, a_0, k_0):
    
    # Initialize arrays to store results
    a = np.zeros(T+1)
    k = np.zeros(T+1)
    w = np.zeros(T+1)
    r = np.zeros(T+1)
    p = np.zeros(T+1)
    y = np.zeros(T+1)
    c = np.zeros(T+1)
    i = np.zeros(T+1)

    # Set initial values
    a[0] = a_0
    k[0] = k_0
    
    # Simulate the economy
    for t in range(T):
        
        # Update capital stock
        k[t+1] = investment_decision(k[t], r[t], a[t], delta)
        
        # Simulate a shock to productivity
        a[t+1] = productivity_shock(a[t], sigma_b)
        
        # Solve for firm's optimal price and output
        p[t], y[t] = firm_optimization(w[t], a[t])
        
        # Solve for household's optimal consumption and investment
        c[t], i[t] = household_optimization(w[t], r[t], pi[t], a[t])
        
        # Determine real wage and real interest rate
        w[t+1] = (1 - alpha) * y[t+1] / labor_supply(w[t], r[t], p[t], y[t], a[t+1])
        r[t+1] = (1 / beta) * (1 + pi[t+1]) / (1 + pi[t]) - 1
        
        # Determine inflation rate
        pi[t+1] = theta * pi[t] + (1 - theta) * phi + gamma * (y[t+1] - 1)
        
    # Return simulation results
    return a, k, w, r, p, y, c, i, pi

In [61]:
import matplotlib.pyplot as plt

def simulate_economy(T, alpha, beta, gamma, delta, sigma_a, sigma_b, pi, a_0, k_0):

    # Run the simulation to generate the results
    y, pi, c, i, w, r = run_simulation(T, alpha, beta, gamma, delta, sigma_a, sigma_b, pi, a_0, k_0)

    # Plot the results
    fig, axs = plt.subplots(3, 2, figsize=(12, 10))

    axs[0, 0].plot(y)
    axs[0, 0].set_title('Output')
    axs[0, 0].set(xlabel='Time', ylabel='Level')

    axs[0, 1].plot(pi)
    axs[0, 1].set_title('Inflation')
    axs[0, 1].set(xlabel='Time', ylabel='Level')

    axs[1, 0].plot(c)
    axs[1, 0].set_title('Consumption')
    axs[1, 0].set(xlabel='Time', ylabel='Level')

    axs[1, 1].plot(i)
    axs[1, 1].set_title('Investment')
    axs[1, 1].set(xlabel='Time', ylabel='Level')

    axs[2, 0].plot(w)
    axs[2, 0].set_title('Real Wage')
    axs[2, 0].set(xlabel='Time', ylabel='Level')

    axs[2, 1].plot(r)
    axs[2, 1].set_title('Real Interest Rate')
    axs[2, 1].set(xlabel='Time', ylabel='Level')

    plt.tight_layout()
    plt.show()
